In [1]:
from pyspark.sql import SparkSession

In [2]:
import os
# Set JAVA_HOME to the correct Java installation path
os.environ['JAVA_HOME'] = '/opt/homebrew/Cellar/openjdk@17/17.0.17/libexec/openjdk.jdk/Contents/Home'
# Create SparkSession
spark = SparkSession.builder.appName("Spark SQL Query Dataframes").getOrCreate()
data_path = '/Users/zlatavorobeva/Documents/GitHub/SQL Int/SQL_Intermediate/Spark/Data'

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/22 19:24:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/22 19:24:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/12/22 19:24:25 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/12/22 19:24:25 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
json_df2_path = data_path + "/utilization.json"
df_util = spark.read.format("json").load(json_df2_path)

In [4]:
df_util.createOrReplaceTempView("utilization")

In [5]:
spark.sql('SELECT server_id, min(cpu_utilization), max(cpu_utilization), stddev(cpu_utilization) \
           FROM utilization \
           GROUP BY server_id').show()

+---------+--------------------+--------------------+-----------------------+
|server_id|min(cpu_utilization)|max(cpu_utilization)|stddev(cpu_utilization)|
+---------+--------------------+--------------------+-----------------------+
|      103|                0.56|                0.96|    0.11617507884178278|
|      104|                0.51|                0.91|    0.11521679513850511|
|      106|                0.22|                0.62|    0.11531539914568226|
|      100|                0.27|                0.67|     0.1152264191787964|
|      105|                0.29|                0.69|    0.11510721467869486|
|      101|                 0.6|                 1.0|    0.11651726263197697|
|      102|                0.56|                0.96|    0.11549678751286807|
|      112|                0.52|                0.92|    0.11528867845082576|
|      113|                0.58|                0.98|    0.11544345150353687|
|      110|                0.35|                0.75|    0.11533

In [6]:
sql_window = "SELECT event_datetime, server_id, cpu_utilization,  \
         avg(cpu_utilization) OVER (PARTITION BY server_id) avg_server_util \
         FROM  \
         utilization"

In [7]:
spark.sql(sql_window).show()

+-------------------+---------+---------------+------------------+
|     event_datetime|server_id|cpu_utilization|   avg_server_util|
+-------------------+---------+---------------+------------------+
|03/05/2019 08:06:31|      110|           0.68|0.5537749999999892|
|03/05/2019 08:11:31|      110|           0.58|0.5537749999999892|
|03/05/2019 08:16:31|      110|           0.55|0.5537749999999892|
|03/05/2019 08:21:31|      110|           0.63|0.5537749999999892|
|03/05/2019 08:26:31|      110|           0.63|0.5537749999999892|
|03/05/2019 08:31:31|      110|           0.71|0.5537749999999892|
|03/05/2019 08:36:31|      110|           0.67|0.5537749999999892|
|03/05/2019 08:41:31|      110|           0.55|0.5537749999999892|
|03/05/2019 08:46:31|      110|           0.37|0.5537749999999892|
|03/05/2019 08:51:31|      110|            0.7|0.5537749999999892|
|03/05/2019 08:56:31|      110|           0.67|0.5537749999999892|
|03/05/2019 09:01:31|      110|           0.56|0.5537749999999

In [11]:
sql_window2 = "SELECT event_datetime, server_id, cpu_utilization,  \
         avg(cpu_utilization) OVER (PARTITION BY server_id) avg_server_util, \
         cpu_utilization - avg(cpu_utilization) OVER (PARTITION BY server_id) AS avg_server_util delta_server_util \
         FROM utilization"

In [13]:
spark.sql(sql_window2).show()

+-------------------+---------+---------------+------------------+--------------------+
|     event_datetime|server_id|cpu_utilization|   avg_server_util|   delta_server_util|
+-------------------+---------+---------------+------------------+--------------------+
|03/05/2019 08:06:31|      110|           0.68|0.5537749999999892|  0.1262250000000108|
|03/05/2019 08:11:31|      110|           0.58|0.5537749999999892|0.026225000000010712|
|03/05/2019 08:16:31|      110|           0.55|0.5537749999999892|-0.00377499999998...|
|03/05/2019 08:21:31|      110|           0.63|0.5537749999999892| 0.07622500000001076|
|03/05/2019 08:26:31|      110|           0.63|0.5537749999999892| 0.07622500000001076|
|03/05/2019 08:31:31|      110|           0.71|0.5537749999999892| 0.15622500000001072|
|03/05/2019 08:36:31|      110|           0.67|0.5537749999999892| 0.11622500000001079|
|03/05/2019 08:41:31|      110|           0.55|0.5537749999999892|-0.00377499999998...|
|03/05/2019 08:46:31|      110| 

In [15]:
sql_window3 = "SELECT event_datetime, server_id, cpu_utilization,  \
                      avg(cpu_utilization) OVER (PARTITION BY server_id ORDER BY event_datetime \
                                    ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) avg_server_util \
                FROM  \
                      utilization"


In [ ]:
spark.sql(sql_window3).show()

+-------------------+---------+---------------+-------------------+
|     event_datetime|server_id|cpu_utilization|    avg_server_util|
+-------------------+---------+---------------+-------------------+
|03/05/2019 08:06:31|      110|           0.68|               0.63|
|03/05/2019 08:11:31|      110|           0.58| 0.6033333333333334|
|03/05/2019 08:16:31|      110|           0.55| 0.5866666666666666|
|03/05/2019 08:21:31|      110|           0.63| 0.6033333333333334|
|03/05/2019 08:26:31|      110|           0.63| 0.6566666666666666|
|03/05/2019 08:31:31|      110|           0.71| 0.6699999999999999|
|03/05/2019 08:36:31|      110|           0.67| 0.6433333333333333|
|03/05/2019 08:41:31|      110|           0.55| 0.5300000000000001|
|03/05/2019 08:46:31|      110|           0.37|               0.54|
|03/05/2019 08:51:31|      110|            0.7|               0.58|
|03/05/2019 08:56:31|      110|           0.67| 0.6433333333333334|
|03/05/2019 09:01:31|      110|           0.56| 

25/12/22 21:13:47 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 276241 ms exceeds timeout 120000 ms
25/12/22 21:13:47 WARN SparkContext: Killing executors is not supported by current scheduler.
25/12/22 21:13:53 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:81)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:669)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1296)
	at o

In [ ]:
(0.71+0.78)/2

In [ ]:
(0.71+0.78+0.87)/3